In [1]:
import vtk
import numpy as np
import gmsh
import math
import random
import os


class CalcMesh:

    def __init__(self, nodes_coords, tetrs_points):
        
        self.nodes = np.array([nodes_coords[0::3],nodes_coords[1::3],nodes_coords[2::3]])

        self.stretch = np.zeros(shape=(int(len(nodes_coords) / 3)), dtype=np.double)

        self.velocity = np.zeros(shape=(3, int(len(nodes_coords) / 3)), dtype=np.double)
        
        for i in range(len(self.velocity[0])):
            r = math.sqrt(math.pow(self.nodes[0,i],2) + math.pow(self.nodes[1,i],2) + math.pow(self.nodes[2,i],2))
            theta = math.acos(self.nodes[2,i] / r)
            phi = math.atan(self.nodes[1,i] / self.nodes[0,i])
            velocity = 5
            self.velocity[0, i] += velocity * math.cos(phi) * math.sin(theta)
            self.velocity[1, i] += velocity * math.sin(phi) * math.sin(theta)
            self.velocity[2, i] += velocity * math.cos(theta)

        self.tetrs = np.array([tetrs_points[0::4],tetrs_points[1::4],tetrs_points[2::4],tetrs_points[3::4]])
        self.tetrs -= 1

    def move(self, tau):
        print(tau, ": ")
        for j in range(len(self.velocity[0])):
            if j!= 0:
                r0 = math.sqrt(math.pow(self.nodes[0,j - 1],2) + math.pow(self.nodes[1,j - 1],2) + math.pow(self.nodes[2,j - 1],2))
            else:
                n = len(self.nodes[0])
                r0 = math.sqrt(math.pow(self.nodes[0,n - 1],2) + math.pow(self.nodes[1,n - 1],2) + math.pow(self.nodes[2,n - 1],2))
            r = math.sqrt(math.pow(self.nodes[0,j],2) + math.pow(self.nodes[1,j],2) + math.pow(self.nodes[2,j],2))
            accelerate = 0.05 * (math.pow(self.velocity[0,j], 2) + math.pow(self.velocity[1,j], 2) + math.pow(self.velocity[2,j], 2))
            velocity = math.sqrt(math.pow(self.velocity[0,j],2) + math.pow(self.velocity[1,j],2) + math.pow(self.velocity[2,j],2))
            theta = math.acos(self.velocity[2,j] / velocity)
            phi = math.atan(self.velocity[1,j] / self.velocity[0,j])
            self.velocity[0,j] -= accelerate * math.cos(phi) * math.sin(theta) * tau
            self.velocity[1,j] -= accelerate * math.sin(phi) * math.sin(theta) * tau
            self.velocity[2,j] -= accelerate * math.cos(theta) * tau
            self.nodes[0,j] += self.velocity[0,j] * tau - accelerate * math.cos(phi) * math.sin(theta) * math.pow(tau, 2) / 2
            self.nodes[1,j] += self.velocity[1,j] * tau - accelerate * math.sin(phi) * math.sin(theta) * math.pow(tau, 2) / 2
            self.nodes[2,j] += self.velocity[2,j] * tau - accelerate * math.cos(theta) * math.pow(tau, 2) / 2
            if (j != 0):
                self.stretch[j] = (r - r0) - self.stretch[j]
            else:
                self.stretch[j] = 0



    def snapshot(self, snap_number):
        unstructuredGrid = vtk.vtkUnstructuredGrid()
        points = vtk.vtkPoints()

        stretch = vtk.vtkDoubleArray()
        stretch.SetName("movement")
        
        direction = vtk.vtkDoubleArray()
        direction.SetNumberOfComponents(3)
        direction.SetName("direction")

        for i in range(0, len(self.nodes[0])):
            velocity = math.sqrt(math.pow(self.velocity[0,i],2) + math.pow(self.velocity[1,i],2) + math.pow(self.velocity[2,i],2))
            points.InsertNextPoint(self.nodes[0,i], self.nodes[1,i], self.nodes[2,i])
            stretch.InsertNextValue(self.stretch[i])
            direction.InsertNextTuple((self.velocity[0,i] / velocity * 100, self.velocity[1,i] / velocity * 100, self.velocity[2,i] / velocity * 100))

        unstructuredGrid.SetPoints(points)
        unstructuredGrid.GetPointData().AddArray(stretch)
        unstructuredGrid.GetPointData().AddArray(direction)

        for i in range(0, len(self.tetrs[0])):
            tetr = vtk.vtkTetra()
            for j in range(0, 4):
                tetr.GetPointIds().SetId(j, self.tetrs[j,i])
            unstructuredGrid.InsertNextCell(tetr.GetCellType(), tetr.GetPointIds())

        writer = vtk.vtkXMLUnstructuredGridWriter()
        writer.SetInputDataObject(unstructuredGrid)
        writer.SetFileName("VaseDetonation-step-" + str(snap_number) + ".vtu")
        writer.Write()


        
gmsh.initialize()

gmsh.merge('Vase.stl')

gmsh.model.mesh.classifySurfaces(30 * math.pi / 180., True, False, math.pi)
gmsh.model.mesh.createGeometry()

entities = gmsh.model.getEntities(2)
loops = gmsh.model.geo.addSurfaceLoop([entities[i][1] for i in range(len(entities))])
gmsh.model.geo.addVolume([loops])

gmsh.model.geo.synchronize()

evaluation = gmsh.model.mesh.field.add("MathEval")
gmsh.model.mesh.field.setString(evaluation, "F", "4")
gmsh.model.mesh.field.setAsBackgroundMesh(evaluation)

gmsh.model.mesh.generate(3)

nodeTags, nodesCoord, parametricCoord = gmsh.model.mesh.getNodes()

GMSH_TETR_CODE = 4
tetrsNodesTags = None
elementTypes, elementTags, elementNodeTags = gmsh.model.mesh.getElements()
for i in range(0, len(elementTypes)):
    if elementTypes[i] != GMSH_TETR_CODE:
        continue
    tetrsNodesTags = elementNodeTags[i]

if tetrsNodesTags is None:
    print("Can not find tetra data. Exiting.")
    gmsh.finalize()
    exit(-2)

print("The model has %d nodes and %d tetrs" % (len(nodeTags), len(tetrsNodesTags) / 4))

for i in range(0, len(nodeTags)):
    assert (i == nodeTags[i] - 1)
assert(len(tetrsNodesTags) % 4 == 0)

mesh = CalcMesh(nodesCoord, tetrsNodesTags)
for i in range(50):
    mesh.move(i)
    mesh.snapshot(i)

gmsh.finalize()

The model has 53127 nodes and 212151 tetrs
0 : 
1 : 
2 : 
3 : 
4 : 
5 : 
6 : 
7 : 
8 : 
9 : 
10 : 
11 : 
12 : 
13 : 
14 : 
15 : 
16 : 
17 : 
18 : 
19 : 
20 : 
21 : 
22 : 
23 : 
24 : 
25 : 
26 : 
27 : 
28 : 
29 : 
30 : 
31 : 
32 : 
33 : 
34 : 
35 : 
36 : 
37 : 
38 : 
39 : 
40 : 
41 : 
42 : 
43 : 
44 : 
45 : 
46 : 
47 : 
48 : 
49 : 
